In [1]:
import numpy as np
import os
from scipy.spatial.transform import Rotation as R
from scipy.spatial.transform import Slerp
from collections import defaultdict

# Pred

### Pred interpolation functions

In [9]:
import os
import numpy as np
from collections import defaultdict
from scipy.spatial.transform import Rotation as R, Slerp

def load_pose(pose_path):
    with open(pose_path, 'r') as f:
        line = f.readline()
    pose = np.array(list(map(float, line.strip().split())))
    assert pose.shape == (16,), f"Pose shape error: {pose.shape}"
    T = pose.reshape(4, 4)
    # 轻微数值修正：强制最后一行 [0,0,0,1]
    T[3, :] = np.array([0.0, 0.0, 0.0, 1.0])
    return T

def parse_start_end_from_filename(filename):
    # 假设形如 "0000_to_0010_*.txt" 或 "0000_..._0010_..." 的风格
    # 你原来的 parts[0], parts[3] 太易碎，这里更稳妥一点：
    name = os.path.basename(filename)
    parts = name.split('_')
    # 尝试从字符串里抓两个整数（从左到右的前两个）
    nums = []
    for p in parts:
        try:
            nums.append(int(p))
        except ValueError:
            pass
        if len(nums) == 2:
            break
    assert len(nums) == 2, f"Cannot parse start/end from filename: {name}"
    return nums[0], nums[1]

def invert_se3(T):
    """SE(3)逆：T = [R t; 0 1] -> T^{-1} = [R^T, -R^T t; 0 1]"""
    T_inv = np.eye(4, dtype=T.dtype)
    Rm = T[:3, :3]
    tm = T[:3, 3]
    Rmt = Rm.T
    T_inv[:3, :3] = Rmt
    T_inv[:3, 3] = -Rmt @ tm
    return T_inv

def interpolate_pose_SE3(T, steps):
    """
    将总体相对位姿 T 在 [0,1] 上做等间隔插值，输出 steps 个增量（每个都是 4x4）。
    旋转用四元数 SLERP，平移线性插值。
    """
    rel_poses = []
    R_total = R.from_matrix(T[:3, :3])
    t_total = T[:3, 3]

    key_times = [0.0, 1.0]
    key_rots = R.from_quat(np.vstack([np.array([0, 0, 0, 1]), R_total.as_quat()]))
    slerp = Slerp(key_times, key_rots)

    for i in range(1, steps + 1):
        ratio = i / steps
        interp_rot = slerp([ratio])[0].as_matrix()
        interp_t = t_total * ratio
        T_i = np.eye(4)
        T_i[:3, :3] = interp_rot
        T_i[:3, 3] = interp_t
        rel_poses.append(T_i)
    return rel_poses

def average_quaternions(quats):
    """
    Markley 方法（特征分解）做四元数平均，已天然处理对顶号。
    quats: (N,4), wxyz 或 xyzw 都无所谓，只要一致；scipy 默认 xyzw。
    你这里使用的是 scipy 的 xyzw，所以继续用 xyzw。
    """
    A = np.zeros((4, 4))
    for q in quats:
        q = q / np.linalg.norm(q)
        A += np.outer(q, q)
    A /= len(quats)
    eigvals, eigvecs = np.linalg.eigh(A)
    return eigvecs[:, np.argmax(eigvals)]

def get_dense_relative_poses(pred_files):
    """
    对于每个跨帧 txt（可能正向也可能反向），把整体相对位姿均匀拆成若干个 (k, k+1) 的候选，
    然后对同一个 (k, k+1) 的多份候选做平移均值 + 四元数平均，得到稠密相对位姿字典。
    返回: dict[(i, i+1)] = 4x4 相对位姿
    """
    pose_candidates = defaultdict(list)

    for f in pred_files:
        T = load_pose(f)
        start, end = parse_start_end_from_filename(f)
        if start == end:
            # 无跨度，跳过
            continue

        # 规范化为“从小到大”的方向
        if end > start:
            steps = end - start
            interp_rel_poses = interpolate_pose_SE3(T, steps)
            base = start
        else:
            steps = start - end
            T_inv = invert_se3(T)          # 从 end -> start 的整体位姿
            interp_rel_poses = interpolate_pose_SE3(T_inv, steps)
            base = end

        # 写入 (k, k+1) 候选
        for i in range(steps):
            key = (base + i, base + i + 1)
            pose_candidates[key].append(interp_rel_poses[i])

    # 聚合候选 -> 平均
    dense_rel_poses = {}
    for k, pose_list in pose_candidates.items():
        # 平移均值
        translations = np.stack([p[:3, 3] for p in pose_list], axis=0)
        avg_t = translations.mean(axis=0)

        # 旋转平均（scipy 是 xyzw）
        rots = np.stack([p[:3, :3] for p in pose_list], axis=0)
        quats = R.from_matrix(rots).as_quat()  # (N,4) xyzw
        avg_quat = average_quaternions(quats)
        avg_rot = R.from_quat(avg_quat).as_matrix()

        T_avg = np.eye(4)
        T_avg[:3, :3] = avg_rot
        T_avg[:3, 3]  = avg_t
        dense_rel_poses[k] = T_avg

    return dense_rel_poses


In [10]:
def load_pose(pose_path):
    with open(pose_path, 'r') as f:
        line = f.readline()
    pose = np.array(list(map(float, line.strip().split())))
    assert pose.shape == (16,), f"Pose shape error: {pose.shape}"
    return pose.reshape(4, 4)

def parse_start_end_from_filename(filename):
    name = os.path.basename(filename)
    parts = name.split('_')
    return int(parts[0]), int(parts[3])

def interpolate_pose(T, steps):
    rel_poses = []
    R_total = R.from_matrix(T[:3, :3])
    t_total = T[:3, 3]
    times = [0, 1]
    slerp = Slerp(times, R.from_quat([[0, 0, 0, 1], R_total.as_quat()]))
    for i in range(1, steps + 1):
        ratio = i / steps
        interp_rot = slerp([ratio])[0].as_matrix()
        interp_t = t_total * ratio
        T_i = np.eye(4)
        T_i[:3, :3] = interp_rot
        T_i[:3, 3] = interp_t
        rel_poses.append(T_i)
    return rel_poses

def average_quaternions(quats):
    A = np.zeros((4, 4))
    for q in quats:
        q = q / np.linalg.norm(q)
        A += np.outer(q, q)
    A /= len(quats)
    eigvals, eigvecs = np.linalg.eigh(A)
    return eigvecs[:, np.argmax(eigvals)]

def get_dense_relative_poses(pred_files):
    # create a dictionary to save {interval} poses for each files, so we have n files, we will have n*interval poses
    # for each interval, we will have n poses

    pose_candidates = defaultdict(list)
    for f in pred_files:
        T = load_pose(f)
        start, end = parse_start_end_from_filename(f)
        interval = end - start
        interp_rel_poses = interpolate_pose(T, interval)
        
        for i in range(interval):
            # 使用 (start + i, start + i + 1) 作为 key
            key = (start + i, start + i + 1)
            pose_candidates[key].append(interp_rel_poses[i])

    dense_rel_poses = {}
    for k, pose_list in pose_candidates.items():
        translations = np.stack([p[:3, 3] for p in pose_list])
        avg_t = translations.mean(axis=0)
        quats = R.from_matrix([p[:3, :3] for p in pose_list]).as_quat()
        avg_quat = average_quaternions(quats)
        avg_rot = R.from_quat(avg_quat).as_matrix()

        T_avg = np.eye(4)
        T_avg[:3, :3] = avg_rot
        T_avg[:3, 3] = avg_t
        dense_rel_poses[k] = T_avg
    return dense_rel_poses


In [25]:
Pose_dir = '/Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Output'

### Pred interpolate pose

In [28]:
import glob
# read traj from /Datasets/Outputs_Feast_compare/Pose_Output/{model}/{video}/{}.txt

pose_root = Pose_dir
video_list = ['trans_t4_b', 'sigmoid_t3_b', 'desc_t4_a', 'cecum_t4_b']

model_list = sorted([
    d for d in os.listdir(pose_root)
    if os.path.isdir(os.path.join(pose_root, d))
])
# model_list = pose_root

for model in model_list:
    print(f"Processing model: {model}")
    for video in video_list:
        model_path = os.path.join(pose_root, model)
        pred_files = sorted(glob.glob(os.path.join(model_path, video, '*_to_*.txt')))
        dense_rel_poses_dict = get_dense_relative_poses(pred_files)
        # print dense_rel_poses_dict shape
        print(f"model: {model}, video: {video}, dense_rel_poses_dict shape: {len(dense_rel_poses_dict)}")
        # save dense_rel_poses_dict to {model_path}/{video}/dense_rel_poses.txt
        # save in text without [] 
        save_path = os.path.join(model_path, video, '_rel_poses.txt') 
        with open(save_path, 'w') as f:
            for k, v in dense_rel_poses_dict.items():
                # print(k[0], k[1], v[0, 0], v[0, 1], v[0, 2], v[0, 3], v[1, 0], v[1, 1], v[1, 2], v[1, 3], v[2, 0], v[2, 1], v[2, 2], v[2, 3], v[3, 0], v[3, 1], v[3, 2], v[3, 3])
                f.write(f"{v[0, 0]} {v[0, 1]} {v[0, 2]} {v[0, 3]} {v[1, 0]} {v[1, 1]} {v[1, 2]} {v[1, 3]} {v[2, 0]} {v[2, 1]} {v[2, 2]} {v[2, 3]} {v[3, 0]} {v[3, 1]} {v[3, 2]} {v[3, 3]}\n")
        print(f"Saved dense relative poses to {save_path}")


Processing model: iid_sfm
model: iid_sfm, video: trans_t4_b, dense_rel_poses_dict shape: 596
Saved dense relative poses to /Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Output/iid_sfm/trans_t4_b/_rel_poses.txt
model: iid_sfm, video: sigmoid_t3_b, dense_rel_poses_dict shape: 535
Saved dense relative poses to /Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Output/iid_sfm/sigmoid_t3_b/_rel_poses.txt
model: iid_sfm, video: desc_t4_a, dense_rel_poses_dict shape: 147
Saved dense relative poses to /Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Output/iid_sfm/desc_t4_a/_rel_poses.txt
model: iid_sfm, video: cecum_t4_b, dense_rel_poses_dict shape: 424
Saved dense relative poses to /Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Output/iid_sfm/cecum_t4_b/_rel_poses.txt
Processing model: monodepth2
model: monodepth2, video: trans_t4_b, dense_rel_poses_dict shape: 596
Saved dense relative poses to /Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Output/monodepth2/tra

#### check pred interpolation

In [ ]:
pred_root = '/Datasets/Outputs_Feast_NEW/Pose_Output'
video_list = ['trans_t4_b']#, 'sigmoid_t3_b', 'desc_t4_a', 'cecum_t4_b']
model_list = sorted([
    d for d in os.listdir(pred_root)
    if os.path.isdir(os.path.join(pred_root, d))
])

for model in model_list:
    for video in video_list:
        pred_files = os.path.join(pred_root, model, video, '_rel_poses.txt')
        pred_rel_poses = load_pred_pose(pred_files)
        print(pred_rel_poses[0])


### Pred loading functions

In [14]:
def load_pred_pose(pred_file):
    poses = []
    with open(pred_file, 'r') as f:
        for line in f:
            values = list(map(float, line.strip().split()))
            matrix = np.array(values).reshape(4, 4)
            poses.append(matrix)
    return np.array(poses)

In [15]:
import matplotlib.pyplot as plt

def get_absolute_traj(first_pose, rel_poses):
    current = first_pose.copy()
    poses = [current.copy()]
    for rel in rel_poses:
        current = current @ rel
        poses.append(current.copy())
    return np.array([p[:3, 3] for p in poses]), np.array(poses)

def visualize_trajectory(trajectories, title):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    for traj in trajectories:
        ax.plot(traj[:, 0], traj[:, 1], traj[:, 2])
        # give two different colors to the two trajectories
        ax.scatter(traj[0, 0], traj[0, 1], traj[0, 2], c='r', marker='o')
        ax.scatter(traj[-1, 0], traj[-1, 1], traj[-1, 2], c='g', marker='o')
    ax.set_title(title)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.view_init(elev=30, azim=45)
    plt.show()

def visualize_trajectory_multi_view(trajectories, title, views=None, save=False, prefix="view"):
    if views is None:
        views = [(30, 45), (60, 45), (30, 135), (90, 0)]
        # views = [(30, i) for i in range(0, 360, 45)]

    for idx, (elev, azim) in enumerate(views):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        for traj in trajectories:
            ax.plot(traj[:, 0], traj[:, 1], traj[:, 2])
            ax.scatter(traj[0, 0], traj[0, 1], traj[0, 2], c='r', marker='o')
            ax.scatter(traj[-1, 0], traj[-1, 1], traj[-1, 2], c='g', marker='o')

        ax.set_title(f"{title} (elev={elev}, azim={azim})")
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.view_init(elev=elev, azim=azim)

        if save:
            fig.savefig(f"{prefix}_e{elev}_a{azim}.png")
        else:
            plt.show()


In [7]:
pred_root = Pose_dir
video_list = ['trans_t4_b']#, 'sigmoid_t3_b', 'desc_t4_a', 'cecum_t4_b']
model_list = sorted([
    d for d in os.listdir(pred_root)
    if os.path.isdir(os.path.join(pred_root, d))
])

for model in model_list:
    for video in video_list:
        pred_files = os.path.join(pred_root, model, video, '_rel_poses.txt')
        pred_rel_poses = load_pred_pose(pred_files)


NameError: name 'load_pred_pose' is not defined

# Evo 

### KITTI format

In [32]:
## for each model, video, take "/Datasets/Outputs_Feast_all/Pose_Output_all/c3vd/{model}/{video}/_rel_poses.txt", and take the first 12 columns and save as "/Datasets/Outputs_Feast_all/Pose_Output_all/c3vd/{model}/{video}/_rel_poses_tum.txt"
import os
import numpy as np

root_dir = Pose_dir
print(f"Root directory: {root_dir}")

for model_name in os.listdir(root_dir):
    print(f"Processing model: {model_name}")
    # model_name = os.path.basename(model_path)
    model_path = os.path.join(root_dir, model_name)
    if not os.path.isdir(model_path):
        continue

    for video_name in os.listdir(model_path):
        video_path = os.path.join(model_path, video_name)
        if not os.path.isdir(video_path):
            continue

        input_file = os.path.join(video_path, "_rel_poses.txt")
        output_file = os.path.join(video_path, "_rel_poses_kitti.txt")

        if not os.path.exists(input_file):
            print(f"Missing: {input_file}")
            continue

        try:
            # 加载数据（按空格分隔）
            data = np.loadtxt(input_file)

            # 取前12列
            data_trimmed = data[:, :12]

            # 保存为新文件，格式化保存为空格分隔
            np.savetxt(output_file, data_trimmed, fmt="%.9f", delimiter=" ")
            print(f"Processed: {model_name}/{video_name}")
        except Exception as e:
            print(f"Error processing {input_file}: {e}")

print("All files processed.")


Root directory: /Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Output
Processing model: monodepth2
Processed: monodepth2/sigmoid_t3_b
Processed: monodepth2/trans_t4_b
Processed: monodepth2/cecum_t4_b
Processed: monodepth2/desc_t4_a
Processing model: monovit
Processed: monovit/sigmoid_t3_b
Processed: monovit/trans_t4_b
Processed: monovit/cecum_t4_b
Processed: monovit/desc_t4_a
Processing model: iid_sfm
Processed: iid_sfm/sigmoid_t3_b
Processed: iid_sfm/trans_t4_b
Processed: iid_sfm/cecum_t4_b
Processed: iid_sfm/desc_t4_a
All files processed.


### calculate prediction abs from rel

In [33]:
import os
import numpy as np

root_dir = Pose_dir
gt_pose_root = "/Datasets/C3VD"

for model_name in os.listdir(root_dir):
    model_path = os.path.join(root_dir, model_name)
    if not os.path.isdir(model_path):
        continue

    for video_name in os.listdir(model_path):
        video_path = os.path.join(model_path, video_name)
        if not os.path.isdir(video_path):
            continue

        rel_pose_file = os.path.join(video_path, "_rel_poses.txt")
        abs_pose_file = os.path.join(video_path, "_abs_poses_kitti.txt")
        gt_pose_file = os.path.join(gt_pose_root, video_name, "pose.txt")

        if not os.path.exists(rel_pose_file):
            print(f"Missing: {rel_pose_file}")
            continue
        if not os.path.exists(gt_pose_file):
            print(f"Missing: {gt_pose_file}")
            continue

        try:
            rel_poses = np.loadtxt(rel_pose_file).reshape(-1, 4, 4)
            gt_pose_data = np.loadtxt(gt_pose_file, delimiter=',')
            first_pose = gt_pose_data[0] 
            T_abs = first_pose.reshape(4, 4).T

            abs_poses_list = [T_abs[:3, :].reshape(12)]  # 加入第一帧
            for T_rel in rel_poses:
                T_abs = T_abs @ T_rel
                abs_poses_list.append(T_abs[:3, :].reshape(12))

            abs_poses_array = np.array(abs_poses_list)
            print(f"{video_name}: abs poses count = {len(abs_poses_list)} (expected {rel_poses.shape[0] + 1})")

            np.savetxt(abs_pose_file, abs_poses_array, fmt="%.9f", delimiter=" ")
            print(f"✅ Saved: {abs_pose_file}")

        except Exception as e:
            print(f"❌ Error processing {video_name}: {e}")

print("✅ All done.")


sigmoid_t3_b: abs poses count = 536 (expected 536)
✅ Saved: /Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Output/monodepth2/sigmoid_t3_b/_abs_poses_kitti.txt
trans_t4_b: abs poses count = 597 (expected 597)
✅ Saved: /Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Output/monodepth2/trans_t4_b/_abs_poses_kitti.txt
cecum_t4_b: abs poses count = 425 (expected 425)
✅ Saved: /Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Output/monodepth2/cecum_t4_b/_abs_poses_kitti.txt
desc_t4_a: abs poses count = 148 (expected 148)
✅ Saved: /Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Output/monodepth2/desc_t4_a/_abs_poses_kitti.txt
sigmoid_t3_b: abs poses count = 536 (expected 536)
✅ Saved: /Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Output/monovit/sigmoid_t3_b/_abs_poses_kitti.txt
trans_t4_b: abs poses count = 597 (expected 597)
✅ Saved: /Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Output/monovit/trans_t4_b/_abs_poses_kitti.txt
cecum_t4_b: abs poses count 

### change prediction to tum format

In [34]:
import os
import numpy as np
from scipy.spatial.transform import Rotation as R

input_root = Pose_dir
print(f"Processing input root: {input_root}")
kitti_filename = "_abs_poses_kitti.txt"
tum_filename = "_abs_poses_tum.txt"

for model_name in os.listdir(input_root):
    # model_name = os.path.basename(model_path)
    model_path = os.path.join(root_dir, model_name)
    if not os.path.isdir(model_path):
        continue

    for video_name in os.listdir(model_path):
        video_path = os.path.join(model_path, video_name)
        if not os.path.isdir(video_path):
            continue

        kitti_path = os.path.join(video_path, kitti_filename)
        tum_path = os.path.join(video_path, tum_filename)

        if not os.path.exists(kitti_path):
            print(f"Missing: {kitti_path}")
            continue

        try:
            kitti_poses = np.loadtxt(kitti_path).reshape(-1, 3, 4)
        except Exception as e:
            print(f"Error loading {kitti_path}: {e}")
            continue

        with open(tum_path, "w") as f:
            for idx, pose in enumerate(kitti_poses):
                R_mat = pose[:, :3]
                t_vec = pose[:, 3]
                quat = R.from_matrix(R_mat).as_quat()  # [x, y, z, w]
                timestamp = idx + 1  # 从1开始
                f.write(f"{timestamp:d} {t_vec[0]:.6f} {t_vec[1]:.6f} {t_vec[2]:.6f} "
                        f"{quat[0]:.6f} {quat[1]:.6f} {quat[2]:.6f} {quat[3]:.6f}\n")

        print(f"Saved TUM poses to {tum_path}")

print("All files processed.")


Processing input root: /Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Output
Saved TUM poses to /Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Output/monodepth2/sigmoid_t3_b/_abs_poses_tum.txt
Saved TUM poses to /Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Output/monodepth2/trans_t4_b/_abs_poses_tum.txt
Saved TUM poses to /Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Output/monodepth2/cecum_t4_b/_abs_poses_tum.txt
Saved TUM poses to /Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Output/monodepth2/desc_t4_a/_abs_poses_tum.txt
Saved TUM poses to /Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Output/monovit/sigmoid_t3_b/_abs_poses_tum.txt
Saved TUM poses to /Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Output/monovit/trans_t4_b/_abs_poses_tum.txt
Saved TUM poses to /Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Output/monovit/cecum_t4_b/_abs_poses_tum.txt
Saved TUM poses to /Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Ou

### change gt to tum format

In [15]:
import os
import numpy as np
import glob
from scipy.spatial.transform import Rotation as R

# 根目录
gt_root = '/Datasets/C3VD'
pred_root = Pose_dir

# 获取所有子文件夹路径（即所有video）
video_folders = sorted(glob.glob(os.path.join(gt_root, '*')))
print(f"Found {len(video_folders)} video folders in GT root.")

for video_path in video_folders:
    video_name = os.path.basename(video_path)
    pose_file = os.path.join(gt_root, video_name, 'pose.txt')  # 修正路径为 GT 路径

    if not os.path.exists(pose_file):
        print(f"pose.txt not found for {video_name}, skipping.")
        continue

    try:
        poses = np.loadtxt(pose_file, delimiter=',').reshape(-1, 4, 4)
    except Exception as e:
        print(f"Error loading {pose_file}: {e}")
        continue

    output_kitti = os.path.join(video_path, '_abs_poses_kitti.txt')
    output_tum = os.path.join(video_path, '_abs_poses_tum.txt')

    with open(output_kitti, 'w') as f_kitti, open(output_tum, 'w') as f_tum:
        for idx, pose in enumerate(poses):
            pose = pose.T
            pose_3x4 = pose[:3, :]  # 取前3行
            flat = pose_3x4.flatten()
            f_kitti.write(' '.join([f'{x:.6f}' for x in flat]) + '\n')

            # 转换为 TUM
            R_mat = pose_3x4[:, :3]
            t_vec = pose_3x4[:, 3]
            quat = R.from_matrix(R_mat).as_quat()  # x y z w
            timestamp = idx + 1
            f_tum.write(f"{timestamp} {t_vec[0]:.6f} {t_vec[1]:.6f} {t_vec[2]:.6f} "
                        f"{quat[0]:.6f} {quat[1]:.6f} {quat[2]:.6f} {quat[3]:.6f}\n")

    print(f"Saved KITTI poses to {output_kitti}")
    print(f"Saved TUM poses to {output_tum}")

print("All GT poses processed.")


Found 22 video folders in GT root.
Saved KITTI poses to /Datasets/C3VD/cecum_t1_a/_abs_poses_kitti.txt
Saved TUM poses to /Datasets/C3VD/cecum_t1_a/_abs_poses_tum.txt
Saved KITTI poses to /Datasets/C3VD/cecum_t1_b/_abs_poses_kitti.txt
Saved TUM poses to /Datasets/C3VD/cecum_t1_b/_abs_poses_tum.txt
Saved KITTI poses to /Datasets/C3VD/cecum_t2_a/_abs_poses_kitti.txt
Saved TUM poses to /Datasets/C3VD/cecum_t2_a/_abs_poses_tum.txt
Saved KITTI poses to /Datasets/C3VD/cecum_t2_b/_abs_poses_kitti.txt
Saved TUM poses to /Datasets/C3VD/cecum_t2_b/_abs_poses_tum.txt
Saved KITTI poses to /Datasets/C3VD/cecum_t2_c/_abs_poses_kitti.txt
Saved TUM poses to /Datasets/C3VD/cecum_t2_c/_abs_poses_tum.txt
Saved KITTI poses to /Datasets/C3VD/cecum_t3_a/_abs_poses_kitti.txt
Saved TUM poses to /Datasets/C3VD/cecum_t3_a/_abs_poses_tum.txt
Saved KITTI poses to /Datasets/C3VD/cecum_t4_a/_abs_poses_kitti.txt
Saved TUM poses to /Datasets/C3VD/cecum_t4_a/_abs_poses_tum.txt
Saved KITTI poses to /Datasets/C3VD/cecum

# evo

In [36]:
import os
import subprocess
import zipfile
import json
import csv
import pandas as pd
import numpy as np

pose_root = Pose_dir
gt_root = "/Datasets/C3VD"
print(f"Pose root: {pose_root}")
output_root = os.path.join(Pose_dir, "Results")

os.makedirs(output_root, exist_ok=True)

zip_dir = os.path.join(output_root, "zip")
os.makedirs(zip_dir, exist_ok=True)

csv_path = os.path.join(output_root, "results_baseline.csv")
csv_fields = ["model", "video", "ATE_RMSE", "RTE_RMSE", "RTE_ROT_DEG"]
rows = []

def project_to_so3(R):
    if not np.isfinite(R).all():
        raise ValueError("Non-finite values in R")
    U, _, Vt = np.linalg.svd(R)
    R_proj = U @ Vt
    if np.linalg.det(R_proj) < 0:
        R_proj *= -1
    if not np.isfinite(R_proj).all():
        raise ValueError("SO(3) projection failed to produce finite matrix")
    return R_proj

def fix_kitti_poses(path_in, path_out):
    poses = np.loadtxt(path_in)
    fixed_poses = []
    for i, row in enumerate(poses):
        try:
            T = row.reshape(3, 4)
            R = T[:, :3]
            t = T[:, 3:]
            R_fixed = project_to_so3(R)
            T_fixed = np.hstack([R_fixed, t])
            fixed_poses.append(T_fixed.reshape(-1))
        except Exception as e:
            print(f"[Warning] Skipping frame {i} due to SO(3) error: {e}")
    fixed_poses = np.array(fixed_poses)
    np.savetxt(path_out, fixed_poses, fmt="%.9f", delimiter=" ")

def extract_rmse_from_zip(zip_path):
    try:
        with zipfile.ZipFile(zip_path, 'r') as zipf:
            for name in zipf.namelist():
                if name.endswith("stats.json"):
                    with zipf.open(name) as f:
                        stats = json.load(f)
                        return stats.get("rmse", None)
    except:
        return None
    return None


for model_name in os.listdir(root_dir):
    # model_name = os.path.basename(model_path)
    model_path = os.path.join(root_dir, model_name)
    if not os.path.isdir(model_path):
        continue

    for video_name in os.listdir(model_path):
        video_path = os.path.join(model_path, video_name)
        if not os.path.isdir(video_path):
            continue

        pred_file = os.path.join(video_path, "_abs_poses_tum.txt")
        gt_file = os.path.join(gt_root, video_name, "_abs_poses_tum.txt")
        if not os.path.exists(pred_file) or not os.path.exists(gt_file):
            print(f"Skipping: {pred_file} or {gt_file} missing")
            continue

        # 修复旋转并保存为临时文件
        # pred_file = pred_file.replace("_abs_poses_tum.txt", "_abs_poses_tum_fixed.txt")
        # fix_kitti_poses(pred_file, pred_file)

        base = f"{model_name}_{video_name}"
        ape_zip = os.path.join(zip_dir, f"{base}_ape.zip")
        rpe_zip = os.path.join(zip_dir, f"{base}_rpe.zip")
        rpe_rot_zip = os.path.join(zip_dir, f"{base}_rpe_rot.zip")
        ape_plot = os.path.join(output_root, f"{base}_ape.png")
        rpe_plot = os.path.join(output_root, f"{base}_rpe.png")
        rpe_rot_plot = os.path.join(output_root, f"{base}_rpe_rot.png")

        try:
            subprocess.run([
                "evo_ape", "tum", gt_file, pred_file,
                "--align", "--correct_scale",
                "--save_results", ape_zip,
                "--plot", "--plot_mode", "xyz",
                "--save_plot", ape_plot
            ], check=True)
            subprocess.run([
                "evo_rpe", "tum", gt_file, pred_file,
                "--align", "--correct_scale",
                "--save_results", rpe_zip,
                "--plot", "--plot_mode", "xyz",
                "--save_plot", rpe_plot
            ], check=True)
            subprocess.run([
                "evo_rpe", "tum", gt_file, pred_file,
                "--align", "--correct_scale",
                "--pose_relation", "angle_deg",
                "--save_results", rpe_rot_zip,
                "--plot", "--plot_mode", "xyz",
                "--save_plot", rpe_rot_plot
            ], check=True)

        except subprocess.CalledProcessError as e:
            print(f"[ERROR] evo failed for {base}: {e}")
            continue

        ate = extract_rmse_from_zip(ape_zip)
        rte = extract_rmse_from_zip(rpe_zip)
        rte_rot = extract_rmse_from_zip(rpe_rot_zip)

        if None in [ate, rte, rte_rot]:
            print(f"[ERROR] Missing values in: {base}")
            continue

        rows.append({
            "model": model_name,
            "video": video_name,
            "ATE_RMSE": round(ate, 6),
            "RTE_RMSE": round(rte, 6),
            "RTE_ROT_DEG": round(rte_rot, 6)
        })

        print(f"✅ {base} — ATE={ate:.3f}, RTE={rte:.3f}, RTE_ROT={rte_rot:.2f}°")

# 保存详细结果表
df = pd.DataFrame(rows)
df.to_csv(csv_path, index=False)
print(f"\n📋 Per-video results saved to: {csv_path}")

# 统一列名为小写，避免groupby失败
df.columns = [str(col).lower() for col in df.columns]

# 按模型聚合
summary = df.groupby("model")[["ate_rmse", "rte_rmse", "rte_rot_deg"]].mean().reset_index()
summary = summary.round(6)
summary_path = os.path.join(output_root, "_summary_by_model_baseline.csv")
summary.to_csv(summary_path, index=False)
print(f"📈 Model-wise summary saved to: {summary_path}")


Pose root: /Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Output
Skipping: /Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Output/Results/zip/_abs_poses_tum.txt or /Datasets/C3VD/zip/_abs_poses_tum.txt missing
APE w.r.t. translation part (m)
(with Sim(3) Umeyama alignment)

       max	8.160588
      mean	3.727458
    median	3.232493
       min	2.018098
      rmse	4.022901
       sse	8674.481218
       std	1.513205

Plot saved to /Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Output/Results/monodepth2_sigmoid_t3_b_ape_raw.png
Plot saved to /Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Output/Results/monodepth2_sigmoid_t3_b_ape_map.png
RPE w.r.t. translation part (m)
for delta = 1 (frames) using consecutive pairs
(with Sim(3) Umeyama alignment)

       max	0.266349
      mean	0.099388
    median	0.089199
       min	0.013431
      rmse	0.111346
       sse	6.632842
       std	0.050197

Plot saved to /Datasets/Outputs_Feast_baseline_pretrained/c3vd_Pose_Output/Re

# Folded

## Prediction

#### Pred pose sanity check

In [ ]:
import numpy as np
path = '/Datasets/Outputs_Feast_compare/Pose_Output/c3vd_mysplit_interval10_supervised_05/cecum_t4_b/0000_color_to_0010_color.txt'
pose = load_pose(path)
print(pose)
start, end = parse_start_end_from_filename(path)
print(start, end)
interval = end - start
interp_rel_poses = interpolate_pose(pose, interval)

pose_candidates = defaultdict(list)
for i in range(interval):
    # 使用 (start + i, start + i + 1) 作为 key
    key = (start + i, start + i + 1)
    pose_candidates[key].append(interp_rel_poses[i])

print(pose_candidates)

### Load pred poses for visualize

## GT

### GT loading functions

In [ ]:
def load_gt_pose(gt_file):
    poses = []
    with open(gt_file, 'r') as f:
        for line in f:
            values = list(map(float, line.strip().split(',')))
            matrix = np.array(values).reshape(4, 4).T
            poses.append(matrix)
    return np.array(poses)

def get_relative_pose(p0, p1):
    return np.linalg.inv(p0) @ p1


### Traj calculation and Visualise

In [ ]:
### calculate trajectories and visualize three angles for each traj
import matplotlib.pyplot as plt

def get_absolute_traj(first_pose, rel_poses):
    current = first_pose.copy()
    poses = [current.copy()]
    for rel in rel_poses:
        current = current @ rel
        poses.append(current.copy())
    return np.array([p[:3, 3] for p in poses]), np.array(poses)

def visualize_trajectory(trajectories, title):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    for traj in trajectories:
        ax.plot(traj[:, 0], traj[:, 1], traj[:, 2])
        # give two different colors to the two trajectories
        ax.scatter(traj[0, 0], traj[0, 1], traj[0, 2], c='r', marker='o')
        ax.scatter(traj[-1, 0], traj[-1, 1], traj[-1, 2], c='g', marker='o')
    ax.set_title(title)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.show()
def visualize_angles(angles, title):
    fig, axs = plt.subplots(3, 1, figsize=(10, 8))
    fig.suptitle(title)
    axs[0].plot(angles[:, 0], label='Roll')
    axs[0].set_ylabel('Roll (rad)')
    axs[1].plot(angles[:, 1], label='Pitch')
    axs[1].set_ylabel('Pitch (rad)')
    axs[2].plot(angles[:, 2], label='Yaw')
    axs[2].set_ylabel('Yaw (rad)')
    plt.xlabel('Frame')
    plt.show()
def get_euler_angles_from_poses(poses):
    angles = []
    for pose in poses:
        r = R.from_matrix(pose[:3, :3])
        euler = r.as_euler('xyz', degrees=False)
        angles.append(euler)
    return np.array(angles)
def visualize_trajectory_and_angles(trajectories, angles, title):
    fig = plt.figure(figsize=(12, 8))
    ax1 = fig.add_subplot(211, projection='3d')
    ax2 = fig.add_subplot(212)
    
    for traj in trajectories:
        ax1.plot(traj[:, 0], traj[:, 1], traj[:, 2])
    
    ax1.set_title(title)
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')

    ax2.plot(angles[:, 0], label='Roll')
    ax2.plot(angles[:, 1], label='Pitch')
    ax2.plot(angles[:, 2], label='Yaw')
    ax2.set_ylabel('Angle (rad)')
    plt.xlabel('Frame')
    plt.legend()
    plt.show()

#### check ground truth recalculation

In [ ]:
gt_root = '/Datasets/C3VD'
video_list = ['trans_t4_b']#, 'sigmoid_t3_b', 'desc_t4_a', 'cecum_t4_b']

for video in video_list:
    gt_pose_path = os.path.join(gt_root, video, 'pose.txt')
    gt_abs_poses = load_gt_pose(gt_pose_path)
    gt_rel_poses = [get_relative_pose(gt_abs_poses[i], gt_abs_poses[i+1]) for i in range(len(gt_abs_poses) - 1)]

    first_pose = gt_abs_poses[0]
    gt_rel_traj, gt_rel_traj_4x4 = get_absolute_traj(first_pose, gt_rel_poses)#gt_abs_poses[0], gt_rel_poses)
    gt_abs_traj = np.array([p[:3, 3] for p in gt_abs_poses])

    # visualize gt_rel_traj and gt_abs_traj
    visualize_trajectory([gt_rel_traj, gt_abs_traj], f"GT Trajectory for {video}")


    gt_angles = get_euler_angles_from_poses(gt_abs_poses)
    visualize_angles(gt_angles, f"GT Angles for {video}")
    gt_rel_angles = get_euler_angles_from_poses(gt_rel_traj_4x4)
    visualize_angles(gt_rel_angles, f"GT Relative Angles for {video}")
    visualize_trajectory_and_angles([gt_rel_traj, gt_abs_traj], gt_angles, f"GT Trajectory and Angles for {video}")


### Traj plots

In [ ]:
### calculate trajectories and visualize three angles for each traj
import matplotlib.pyplot as plt

def get_absolute_traj(first_pose, rel_poses):
    current = first_pose.copy()
    poses = [current.copy()]
    for rel in rel_poses:
        current = current @ rel
        poses.append(current.copy())
    return np.array([p[:3, 3] for p in poses]), np.array(poses)

def visualize_trajectory(trajectories, title):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    for traj in trajectories:
        ax.plot(traj[:, 0], traj[:, 1], traj[:, 2])
        # give two different colors to the two trajectories
        ax.scatter(traj[0, 0], traj[0, 1], traj[0, 2], c='r', marker='o')
        ax.scatter(traj[-1, 0], traj[-1, 1], traj[-1, 2], c='g', marker='o')
    ax.set_title(title)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.view_init(elev=30, azim=45)
    plt.show()

def visualize_trajectory_multi_view(trajectories, title, views=None, save=False, prefix="view"):
    if views is None:
        views = [(30, 45), (60, 45), (30, 135), (90, 0)]
        # views = [(30, i) for i in range(0, 360, 45)]

    for idx, (elev, azim) in enumerate(views):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        for traj in trajectories:
            ax.plot(traj[:, 0], traj[:, 1], traj[:, 2])
            ax.scatter(traj[0, 0], traj[0, 1], traj[0, 2], c='r', marker='o')
            ax.scatter(traj[-1, 0], traj[-1, 1], traj[-1, 2], c='g', marker='o')

        ax.set_title(f"{title} (elev={elev}, azim={azim})")
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.view_init(elev=elev, azim=azim)

        if save:
            fig.savefig(f"{prefix}_e{elev}_a{azim}.png")
        else:
            plt.show()


gt_root = '/Datasets/C3VD'
video_list = ['trans_t4_b']#, 'sigmoid_t3_b', 'desc_t4_a', 'cecum_t4_b']

for video in video_list:
    gt_pose_path = os.path.join(gt_root, video, 'pose.txt')
    gt_abs_poses = load_gt_pose(gt_pose_path)
    gt_rel_poses = [get_relative_pose(gt_abs_poses[i], gt_abs_poses[i+1]) for i in range(len(gt_abs_poses) - 1)]

    first_pose = np.eye(4)
    gt_rel_traj, gt_rel_traj_4x4 = get_absolute_traj(first_pose, gt_rel_poses)#gt_abs_poses[0], gt_rel_poses)
    gt_abs_traj = np.array([p[:3, 3] for p in gt_abs_poses])

    # visualize gt_rel_traj and gt_abs_traj
    # visualize_trajectory([gt_rel_traj], f"GT Trajectory for {video}")

    visualize_trajectory_multi_view(
    trajectories=[gt_abs_traj],
    title="Trajectory GT vs Relative",
    views=[(30, 45), (30, 90), (90, 0), (0, 0)],
    save=False  
    )

    first_pose = gt_abs_poses[0]
    gt_rel_traj, gt_rel_traj_4x4 = get_absolute_traj(first_pose, gt_rel_poses)#gt_abs_poses[0], gt_rel_poses)
    gt_abs_traj = np.array([p[:3, 3] for p in gt_abs_poses])

    # visualize gt_rel_traj and gt_abs_traj
    # visualize_trajectory([gt_rel_traj], f"GT Trajectory for {video}")

    visualize_trajectory_multi_view(
    trajectories=[gt_abs_traj],
    title="Trajectory GT vs Relative",
    views=[(30, 45), (30, 90), (90, 0), (0, 0)], 
    save=False 
    )


### del all created files

In [ ]:
import os

root_dir = "/Datasets/Outputs_Feast_NEW/Pose_Output"
deleted_files = []

for model_name in os.listdir(root_dir):
    model_path = os.path.join(root_dir, model_name)
    if not os.path.isdir(model_path):
        continue

    for video_name in os.listdir(model_path):
        video_path = os.path.join(model_path, video_name)
        if not os.path.isdir(video_path):
            continue

        file_to_delete = os.path.join(video_path, "_abs_poses_kitti.txt")
        if os.path.exists(file_to_delete):
            try:
                os.remove(file_to_delete)
                deleted_files.append(file_to_delete)
            except Exception as e:
                print(f"Failed to delete {file_to_delete}: {e}")

print(f"Deleted {len(deleted_files)} files.")


### change GT to kitti

In [ ]:
## for each video gt under /Datasets/C3VD/{video}/pose.txt, reshape to (4,4).T, and take first 3 rows and flatten into 1*12 with space delimiter, and save as _abs_poses_kitti.txt
import os
import numpy as np
import glob

# 根目录
gt_root = '/Datasets/C3VD'
pred_root = ''/Datasets/Outputs_Feast_ablation/Pose_Outputs_ablation_redid/from_scratch'

# 获取所有子文件夹路径（即所有video）
video_folders = sorted(glob.glob(os.path.join(pred_root, '*')))

for video_path in video_folders:
    video_name = os.path.basename(video_path)
    pose_file = os.path.join(video_path, 'pose.txt')

    if not os.path.exists(pose_file):
        print(f"pose.txt not found in {video_name}, skipping.")
        continue

    # 读取pose.txt，假设是4x4矩阵一行接一行排列
    try:
        poses = np.loadtxt(pose_file, delimiter=',').reshape(-1, 4, 4)
    except Exception as e:
        print(f"Error loading {pose_file}: {e}")
        continue

    # 输出文件路径
    output_file = os.path.join(video_path, '_abs_poses_kitti.txt')
    with open(output_file, 'w') as f:
        for pose in poses:
            pose = pose.T  # 转置
            pose_3x4 = pose[:3, :]  # 取前3行（3x4）
            flat = pose_3x4.flatten()  # 展平为1x12
            flat_str = ' '.join([f'{x:.6f}' for x in flat])  # 格式化为字符串
            f.write(flat_str + '\n')

    print(f"Saved KITTI poses to {output_file}")



## evo

In [ ]:
import os
import subprocess
import zipfile
import json
import csv

pose_root = "/Datasets/Outputs_Feast_NEW/Pose_Output"
gt_root = "/Datasets/C3VD"
output_root = "/Datasets/Outputs_Feast_NEW/Pose_Output/ablation_results"

# 子目录
plot_dir = os.path.join(output_root, "png")
zip_dir = os.path.join(output_root, "zip")
os.makedirs(plot_dir, exist_ok=True)
os.makedirs(zip_dir, exist_ok=True)

csv_path = os.path.join(output_root, "results_tum.csv")
csv_fields = ["Model", "Video", "ATE_RMSE", "RTE_RMSE"]
rows = []

def extract_rmse_from_zip(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zipf:
        for name in zipf.namelist():
            if name.endswith("stats.json"):
                with zipf.open(name) as f:
                    stats = json.load(f)
                    return stats.get("rmse", None)
    return None

for model_name in os.listdir(pose_root):
    model_path = os.path.join(pose_root, model_name)
    if not os.path.isdir(model_path):
        continue

    for video_name in os.listdir(model_path):
        video_path = os.path.join(model_path, video_name)
        if not os.path.isdir(video_path):
            continue

        pred_file = os.path.join(video_path, "_abs_poses_tum.txt")
        gt_file = os.path.join(gt_root, video_name, "_abs_poses_tum.txt")
        if not os.path.exists(pred_file) or not os.path.exists(gt_file):
            print(f"Skipping: {pred_file} or {gt_file} missing")
            continue

        base = f"{model_name}_{video_name}"
        ape_plot = os.path.join(plot_dir, f"{base}_ape.png")
        rpe_plot = os.path.join(plot_dir, f"{base}_rpe.png")

        # Run evo_ape
        try:
            subprocess.run([
                "evo_ape", "tum", gt_file, pred_file,
                "--align",
                "--correct_scale",
                "--plot", "--plot_mode", "xyz",
                "--save_plot", ape_plot
            ], check=True)
        except subprocess.CalledProcessError:
            print(f"Error in APE: {base}")
            continue

        # Run evo_rpe
        try:
            subprocess.run([
                "evo_rpe", "tum", gt_file, pred_file,
                "--align",
                "--correct_scale",
                "--plot", "--plot_mode", "xyz",
                "--save_plot", rpe_plot
            ], check=True)
        except subprocess.CalledProcessError:
            print(f"Error in RPE: {base}")
            continue

        # Extract RMSE
        ate_rmse = extract_rmse_from_zip(ape_zip)
        rte_rmse = extract_rmse_from_zip(rpe_zip)

        if ate_rmse is not None and rte_rmse is not None:
            rows.append({
                "Model": model_name,
                "Video": video_name,
                "ATE_RMSE": f"{ate_rmse:.6f}",
                "RTE_RMSE": f"{rte_rmse:.6f}"
            })
            print(f"✅ {base}: ATE={ate_rmse:.4f}, RTE={rte_rmse:.4f}")
        else:
            print(f"❌ Failed to extract RMSE for: {base}")

# Save CSV summary
with open(csv_path, "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=csv_fields)
    writer.writeheader()
    writer.writerows(rows)

print(f"\n✅ All finished. Results saved to: {csv_path}")


In [ ]:
import os
import subprocess
import zipfile
import json
import csv

pose_root = "/Datasets/Outputs_Feast_all/Pose_Output_all/hk"
gt_root = "/Datasets/C3VD"
output_root = "/Datasets/Outputs_Feast_all/EvoEvalResults_Stable"

# 子目录
plot_dir = os.path.join(output_root, "png")
zip_dir = os.path.join(output_root, "zip")
os.makedirs(plot_dir, exist_ok=True)
os.makedirs(zip_dir, exist_ok=True)

csv_path = os.path.join(output_root, "results.csv")
csv_fields = ["Model", "Video", "ATE_RMSE", "RTE_RMSE"]
rows = []

def extract_rmse_from_zip(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zipf:
        for name in zipf.namelist():
            if name.endswith("stats.json"):
                with zipf.open(name) as f:
                    stats = json.load(f)
                    return stats.get("rmse", None)
    return None

for model_name in os.listdir(pose_root):
    model_path = os.path.join(pose_root, model_name)
    if not os.path.isdir(model_path):
        continue

    for video_name in os.listdir(model_path):
        video_path = os.path.join(model_path, video_name)
        if not os.path.isdir(video_path):
            continue

        pred_file = os.path.join(video_path, "_abs_poses_kitti.txt")
        gt_file = os.path.join(gt_root, video_name, "_abs_poses_kitti.txt")
        if not os.path.exists(pred_file) or not os.path.exists(gt_file):
            print(f"Skipping: {pred_file} or {gt_file} missing")
            continue

        base = f"{model_name}_{video_name}"
        ape_zip = os.path.join(zip_dir, f"{base}_ape.zip")
        rpe_zip = os.path.join(zip_dir, f"{base}_rpe.zip")
        ape_plot = os.path.join(plot_dir, f"{base}_ape.png")
        rpe_plot = os.path.join(plot_dir, f"{base}_rpe.png")

        # Run evo_ape
        try:
            subprocess.run([
                "evo_ape", "kitti", gt_file, pred_file,
                "--align",
                "--correct_scale",
                "--save_results", ape_zip,
                "--plot", "--plot_mode", "xyz",
                "--save_plot", ape_plot
            ], check=True)
        except subprocess.CalledProcessError:
            print(f"Error in APE: {base}")
            continue

        # Run evo_rpe
        try:
            subprocess.run([
                "evo_rpe", "kitti", gt_file, pred_file,
                "--align",
                "--correct_scale",
                "--save_results", rpe_zip,
                "--plot", "--plot_mode", "xyz",
                "--save_plot", rpe_plot
            ], check=True)
        except subprocess.CalledProcessError:
            print(f"Error in RPE: {base}")
            continue

        # Extract RMSE
        ate_rmse = extract_rmse_from_zip(ape_zip)
        rte_rmse = extract_rmse_from_zip(rpe_zip)

        if ate_rmse is not None and rte_rmse is not None:
            rows.append({
                "Model": model_name,
                "Video": video_name,
                "ATE_RMSE": f"{ate_rmse:.6f}",
                "RTE_RMSE": f"{rte_rmse:.6f}"
            })
            print(f"✅ {base}: ATE={ate_rmse:.4f}, RTE={rte_rmse:.4f}")
        else:
            print(f"❌ Failed to extract RMSE for: {base}")

# Save CSV summary
with open(csv_path, "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=csv_fields)
    writer.writeheader()
    writer.writerows(rows)

print(f"\n✅ All finished. Results saved to: {csv_path}")


In [ ]:
import pandas as pd

# 读取原始结果表
input_file = "/Datasets/Outputs_Feast_all/EvoEvalResults_Stable_w_rot/results_hk_tum.csv"  # ← 替换为你的文件路径
df = pd.read_csv(input_file)

# 按模型分组求均值
summary = df.groupby("model")[["ATE_RMSE", "RTE_RMSE", "RTE_ROT_DEG"]].mean().reset_index()

# 保留小数点后6位
summary["ATE_RMSE"] = summary["ATE_RMSE"].round(6)
summary["RTE_RMSE"] = summary["RTE_RMSE"].round(6)
summary["RTE_ROT_DEG"] = summary["RTE_ROT_DEG"].round(6)

# 保存为新的CSV文件
summary_file = "/Datasets/Outputs_Feast_all/EvoEvalResults_Stable_w_rot/summary_by_model_additional.csv"
summary.to_csv(summary_file, index=False)

print(f"✅ 平均结果保存到: {summary_file}")
